# Сравнение цен на сайтах Тираэт и Хайтек

## Содержание

* [Описание проекта](q#01)
  - [Цель проекта](q011)
  - [Постановка задачи](#q012)
  - [Описание данных](#q013)
* [Загрузка необходимых библиотек и функций](#q02)
  - [Инсталяция библиотек](#q021)
  - [Импорт библиотек](#q022)
  - [Выбор параметров](#q023)
  - [Функции используемые в работе](#q024)
* [Загрузка данных](#q03)
* [Парсинг c сайта Тираэт](#q04)
* [Парсинг c сайта Хайтек](#q05)
  - [Группировка товара](#q051)
* [Объединение данных](#q06)
* [Аналитика товаров](#q07)
* [Выгрузка данных](#q08)

## 1.Описание проекта 

### Цель проекта 

Цель данного проекта собрать данные с сайтов https://tiraet.com/ и https://hi-tech.md/ для анвлиза цен в категориях и группах товара

### Постановка задачи 

- загрузить данные с сайта Тираэт
- Загрузить данные с сайта Хайтек
- Объеденить данные в общей таблице
- Добавить дополнительные признаки
- Выгрузить данные в файл для дальнейшей аналитики в tableau

### Описание данных 

- `url` Url адресс товара
- `id` Интендификатор товара
- `_category` Категория товара
- `_subcategory` Подкатегория товара
- `_group` Группа товара
- `brend` Бренд товара
- `title` Название товара
- `date` Дата парсинга
- `price` Цена товара
- `old_price` Предидущая цена товара
- `action_sale` Является ли товар акционным
- `sale` Процент скидки на товар
- `group_prise` Ценовая категория товара
- `dif_price` Отношение максимальной к минимальной цене в группе товаров
- `availability` Наличие товара
- `count_group` Количество товара в группе
- `activ_count_group` Количество товара в группе, который есть в наличии
- `error_group` Наличие ошибки при парсинге товара

##  Загрузка необходимых библиотек и функций

### Инсталяция библиотек

In [ ]:
! pip install matplotlib -q
! pip install matplotlib-inline -q
! pip install missingno -q
! pip install numba -q
! pip install pandas -q
! pip install seaborn -q
! pip install transliterate -q
! pip install ydata-profiling -q

### Импорт библиотек 

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
from ydata_profiling import ProfileReport

### Выбор параметров

In [ ]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
warnings.filterwarnings('ignore')

###   Функции используемые в работе 

In [6]:
#  функция предварительного просмотра данных
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

In [7]:
# функция предварительного парсинга
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'url', 'date', 'availability', 'id', 'price', 'old_price'])    
    for u in url:
        reqx = requests.get(u)
        soup = bs(reqx.text, 'html.parser')
        max_page = soup.findAll('div', class_='nums')
        maxx = int(max_page[0].text.strip().split('\n')[-1])
        for start_position in tqdm(range(1,maxx + 1)):        
            reqx = requests.get(u + str(start_position))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'url' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'date' : pd.to_datetime('today').normalize(),
                             'availability' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN ,
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

In [8]:
#  функция для парсинга данных о группах и категориях товара
def find_groups(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_1 = soup.findAll('a', class_='breadcrumbs__link colored_theme_hover_bg-el-svg')
    elem_2 = soup.findAll('div', class_='breadcrumbs__item')
    try:
        if len(elem_1) <=2:
            if len(elem_2) <= 4:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            else:
                d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
                d_16 = 'T_'+(elem_2[4].text.replace(',','').replace(' ','_').replace('-','_').lower())
        else:
            d_14 = 'T_'+(elem_1[0].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_15 = 'T_'+(elem_1[1].text.replace(',','').replace(' ','_').replace('-','_').lower())
            d_16 = 'T_'+(elem_1[2].text.replace(',','').replace(' ','_').replace('-','_').lower())
    except:
        d_14 = 'err_'
        d_15 = 'err_'
        d_16 = 'err_'
        
    return pd.Series([ (d_14), (d_15), (d_16)])

In [9]:
# функция дляизменения регистра записи на нижний
def lower(x):
    return x.lower()

In [10]:
# Функция создания бренда
def brends(x):
    xx = x.replace('\xa0', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'

In [11]:
# функция подсчета количества товаров в группе
def count_group(x):
    return df[df['_group']==x]['id'].count()

In [12]:
# Функция подсчета количества товаров в группе
def count_group_presence(x):
    return df[(df['availability'].isin(['Мало','Достаточно','Много','В наличии']))&(df['_group']==x)]['id'].count()   

In [13]:
# функция подсчета отношения максимума группы к минимуму
def dif_price(x):    
    max_price = df[(df['availability'].isin(['Мало','Достаточно','Много','В наличии']))&(df['_group']==x)]['price'].max()
    min_price = df[(df['availability'].isin(['Мало','Достаточно','Много','В наличии']))&(df['_group']==x)]['price'].min() 
    return max_price / min_price

In [14]:
# функция создания столбца с кодом товара
def h_find_id(x):
    xxx = x.find('span', class_="ty-control-group__item").text  
    return xxx

In [15]:
# функция создания столбца с парсингом наличия товара
def h_find_availability(t):
    xxx = (t.find('a', class_="ty-control-group product-list-field cm-external-click")
           .text.replace('\ue924','').replace('\ue86c','').replace('\n','').replace('В н','В_н').replace(' ',''))
    return xxx  

In [16]:
# функция создающая столбец с названием товара
def h_find_title(t):
    xxx = t.find('a', class_="product-title").text
    return xxx

In [17]:
# функция создающая столбец с продажной ценой товара
def h_find_price(t):
    xxx = t.find('span', class_="ty-price-num").text.replace(u'\xa0',u'')
    return xxx

In [18]:
# функция созжающая столбец с предидущей ценой для акционного товара
def h_find_old_price(t):
    try:
        xxx = t.find('span', class_="ty-strike").text.replace(u'\xa0',u'').replace(u'руб',u'')
        return xxx
    except:
        return np.NaN

In [19]:
# Функция создающая столбец с урл адресом товара
def h_find_url(t):
    xxx = t.find('a', class_="abt-single-image")['href']
    return xxx

In [20]:
# Функция предварительного парсинга сайта, которая проходит по всем страницам в урл адресах переменной all_url 
# и создает датафрейм в котором храняться: урл адреса товаров, индентефикатор товаров, дата предварительного парсинга товара, 
# наличия товара, названия товара, цены товара, старой цены для акционного товара.
def h_parsing_2(url):
    data = pd.DataFrame(columns=['url', 'id', 'date', 'availability', 'title', 'price', 'old_price'])
    for i in tqdm(url):        
        q = 1
        attempt = 1
        while q>0 or attempt<4:             
            reqx = requests.get(i + str(q))
            soup = bs(reqx.text, 'html.parser')
            datax = pd.DataFrame({'url':pd.Series(soup.findAll('div', class_='ypi-grid-list__item_body'),dtype = 'object')})
            data = pd.concat([data, datax], ignore_index=True)
            if len(list(soup.findAll('div', class_='ypi-grid-list__item_body'))) > 0:
                q = q + 1
                attempt = 1                
            else:
                if attempt == 3:                    
                    q = 0
                    attempt += 1
                else:
                    attempt += 1                    
    data['id'] = data['url'].apply(h_find_id)
    data['date'] = pd.to_datetime('today').normalize()
    data['availability'] = data['url'].apply(h_find_availability)
    data['title'] = data['url'].apply(h_find_title)
    data['price'] = data['url'].apply(h_find_price)
    data['old_price'] = data['url'].apply(h_find_old_price)
    data['url'] = data['url'].apply(h_find_url)
    data['price'] = data['price'].astype(float)
    data.loc[data['old_price'] == '','old_price'] = np.nan
    data['old_price'] = data['old_price'].astype(float)
    return data

In [21]:
# Функция основного парсинга, которая для каждой строки по урл адресу парсит и находит наличие товара
# в магахинах а также группу товара, категорию товара, подкатегорию товата и количество подкатегорий товара
def h_xxx(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_2 = soup.findAll('div', class_='ty-breadcrumbs clearfix')
    name = soup.findAll('span', class_='ty-breadcrumbs__current')
            
    try:
        names = name[0].text
        name_group = elem_2[0].text.replace('/'+str(names),'').replace('\n','')
        err = 'err'
    except:
        names = 'Noname'
        name_group = 'Noname/Noname/Noname/Noname/Noname/Noname'
        err = 'Noname_err'
    try:
        len_name = len((elem_2[0].text.replace('/'+str(names),'').replace('\n','')).split('/'))
    except:
        len_name = 1
        
    try:
        if len_name <= 1:
            d_14 = 'err1'
            d_15 = 'err1'
            d_16 = 'err1'
            d_17 = 'err1'
        elif len_name <= 2:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()      
        elif len_name <= 3:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_17 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name == 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower()
        elif len_name > 4:
            d_14 = 'H_'+(name_group.split("/")[1]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_15 = 'H_'+(name_group.split("/")[2]).replace(',','').replace(' ','_').replace('-','_').lower()
            d_16 = 'H_'+(name_group.split("/")[3]).replace(',','').replace(' ','_').replace('-','_').lower() 
            d_17 = 'H_'+(name_group.split("/")[4]).replace(',','').replace(' ','_').replace('-','_').lower()
    except:
        d_14 = 'err0'
        d_15 = 'err0'
        d_16 = 'err0'
        d_17 = 'err0'
        
    return pd.Series([(d_14), (d_15), (d_16), (d_17)])

## Загрузка данных

In [22]:
brend = pd.read_excel('csv/brend.xlsx')

In [23]:
tiraet_group = pd.read_csv('csv/atiraet_group.csv')

In [24]:
tir_url=['https://tiraet.com/catalog/Komplektujuschie/?PAGEN_1=',
         'https://tiraet.com/catalog/2873/?PAGEN_1=',
         'https://tiraet.com/catalog/Kompjuternaja-mebel/?PAGEN_1=',
         'https://tiraet.com/catalog/bytovaya/?PAGEN_1=',
         'https://tiraet.com/catalog/ohrannye-sistemy/?PAGEN_1=',
         'https://tiraet.com/catalog/svet-i-jelektrika/?PAGEN_1=',
         'https://tiraet.com/catalog/orgtehnika/?PAGEN_1=',
         'https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=',
         'https://tiraet.com/catalog/setevoe-oborudovanie/?PAGEN_1=',
         'https://tiraet.com/catalog/jelektropitanie/?PAGEN_1=',
         'https://tiraet.com/catalog/sport/?PAGEN_1=',
         'https://tiraet.com/catalog/avtojelektronika/?PAGEN_1=',
         'https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=',         
]

In [25]:
start_position=1

In [26]:
hi_url = ['https://hi-tech.md/televizory-i-elektronika/page-',
           'https://hi-tech.md/bytovaya-tehnika/page-',
           'https://hi-tech.md/kompyuternaya-tehnika/page-',
           'https://hi-tech.md/mebel-tekstil/kompyuternaya-mebel/ofisnye-kresla-i-stulya/komp.-kresla/page-',
           'https://hi-tech.md/instrumenty-i-oborudovanie/elektroinstrument/page-',
           'https://hi-tech.md/tovary-dlya-doma/page-'
          ]

переменная со списком ссылок на сайт для парсинга

In [27]:
hai_group = pd.read_csv('csv/ahay_group.csv')

загрузка таблицы с группами товара

## Парсинг c сайта Тираэт

179  -  16  -  23  -  100  -  33  -  60  -  10  -  4  -  31  -  25  -  5  -  5  -  81 

In [28]:
dft = parsing_2(tir_url)

  0%|          | 0/192 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

Загружаем данные из файла  с информацией о группах и категориях товара по его `id`

In [29]:
tiraet_group = tiraet_group.drop_duplicates(keep= False )

Удаляем дублекаты в загруженном фрейме

In [30]:
preprocessing(dft)

таблица имеет следующий вид:


,title,url,date,availability,id,price,old_price
0,Конверт бумажный для CD/DVD на 1 диск Verbatim (49992) 1 шт.,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/konvert_bumazhnyy_dlya_cd_dvd_na_1_disk_verbatim_49992_1_sht/,2024-11-07,Достаточно,88716,1.0,NaN
1,"BOX для CD/DVD Verbatim Slim, на 1 диск 120мм, PVC (49979) Black 1шт",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/box_dlya_cd_dvd_verbatim_slim_na_1_disk_120mm_pvc_49979_black_1sht/,2024-11-07,Много,85078,6.6,NaN
2,Диск CDR 700MB VERBATIM 52X (1шт.) Extra Protection арт.43787,https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/diski/disk_cdr_700mb_verbatim_52x_1sht_extra_protection_art_43787/,2024-11-07,Мало,55722,6.6,NaN
3,"Диск CD-R 700MB VERBATIM, 52X (1шт) EXTRA PROTECTION(43411/43351/43437)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/diski/disk_cdr_700mb_verbatim_52x_1sht_extra_protection_43411_43351_43437/,2024-11-07,Много,12819,6.6,NaN
4,"Диск DVD+R VERBATIM 16X 4,7GB (1шт.) (43550/43551/43500/43498/43488)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/diski/disk_dvd_r_verbatim_16x_4_7gb_1sht_43550_43551_43500_43498_43488/,2024-11-07,Мало,4975,7.4,NaN


****************************************************************************************************
таблица имеет 12190 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


title               0
url                 0
date                0
availability        0
id                  0
price               5
old_price       10217
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


title            0.0
url              0.0
date             0.0
availability     0.0
id               0.0
price            0.0
old_price       83.8
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


title                   object
url                     object
date            datetime64[ns]
availability            object
id                      object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12190 entries, 0 to 12189
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         12190 non-null  object        
 1   url           12190 non-null  object        
 2   date          12190 non-null  datetime64[ns]
 3   availability  12190 non-null  object        
 4   id            12190 non-null  object        
 5   price         12185 non-null  float64       
 6   old_price     1973 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 666.8+ KB


None

In [31]:
dft = dft.merge(tiraet_group, how='left', on='id')

Объединяем датафрейм предварительного парсинга с датафреймом о группах и категориях товара

In [32]:
t_df_new = dft[(dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna())]

Отфильтровываем в отдельный датафрейм данные не содержащие информацию о группах и категориях товара

In [33]:
dft = dft[~((dft['_category'].isna()) | (dft['_subcategory'].isna()) | (dft['_group'].isna()))]

Отчищаем основной датафрейм от данных о товарах не содержащих информацию о группах и категориях

In [34]:
tqdm.pandas() 
t_df_new[['_category','_subcategory','_group']] = t_df_new['url'].progress_apply(find_groups)

  0%|          | 0/2 [00:00<?, ?it/s]

Парсим данные о группах и категориях товара

In [35]:
t_data = t_df_new[~(t_df_new['_category']=='err_')].reset_index(drop=True)
t_data = t_data[['id', '_category','_subcategory','_group' ]]
t_data = t_data.sort_values(['_category','_subcategory', '_group'])

Выделяем информацию о группах и категориях нового товара в отдельную переменную и сортируем ее

In [36]:
tiraet_group = pd.concat([tiraet_group, t_data], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)
tiraet_group.to_csv('csv/atiraet_group.csv', index=False)  

Обединяем информацию о группах и категориях товара в один датафрейм и сохраняем ее в старый файл

In [37]:
dft = pd.concat([dft, t_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)

Объединяем очищенный датафрейм с информацией о товарах и группах товаровс новым датафреймом о товарах и группах товаров

In [38]:
dft[dft['_category']=='err_'].id.count()

0

смотрим количество данных спарсенное с ошибкой

In [39]:
dft[dft['_category']=='err_']

,title,url,date,availability,id,price,old_price,_category,_subcategory,_group


просматриваем данные спарсенные с ошибкой

In [40]:
dft['error_group'] = 0
dft.loc[dft['_category']=='err', 'error_group'] = 1

В датафрейме создаем столбец данные в котором равны "1", если данные о группе товаров спарсены с ошибкой

In [41]:
for i in dft['_category'].unique():
    t = dft[dft['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

T_автотовары
['T_автомагнитола' 'T_авторегистратор' 'T_автохолодильник'
 'T_аксессуары_для_авто' 'T_динамики' 'T_компрессор' 'T_компрессоры'
 'T_крепления_смартфонов/планшетов' 'T_модулятор_fm' 'T_модуляторы_fm'
 'T_навигаторы_gps']
----------------------------------------------------------------------------------------------------
T_безопасность
['T_видеонаблюдение' 'T_домофоны' 'T_монтаж_и_питание'
 'T_оборудование_poe' 'T_сигнализация_охранно_пожарная' 'T_скуд'
 'T_умный_дом']
----------------------------------------------------------------------------------------------------
T_бытовая_техника
['T_гигиена_здоровье_уход' 'T_климатическая_техника'
 'T_крупная_бытовая_техника' 'T_мелкая_бытовая_техника' 'T_телевизоры'
 'T_техника_для_кухни' 'T_часы_наручные']
----------------------------------------------------------------------------------------------------
T_для_дома
['T_бытовая_химия' 'T_запасные_части' 'T_инструменты' 'T_крепления'
 'T_кухонная_утварь' 'T_мебель' 'T_текстиль' 'T_хо

In [42]:
dft['tir'] = 1

## Парсинг c сайта Хайтек

In [43]:
dfh = h_parsing_2(hi_url)

  0%|          | 0/6 [00:00<?, ?it/s]

предварительный парсинг

In [44]:
dfh = dfh.drop_duplicates().reset_index(drop=True)

удаление дублекатов

In [45]:
preprocessing(dfh)

таблица имеет следующий вид:


,url,id,date,availability,title,price,old_price
0,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr1220-01/,Т-000088228,2024-11-07,В_наличии,Батарейка Energenie EG-BA-CR1220-01,4.0,NaN
1,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2016-01/,Т-000088229,2024-11-07,В_наличии,Батарейка Energenie EG-BA-CR2016-01,4.0,NaN
2,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aa4-01/,Т-000088221,2024-11-07,В_наличии,Батарейка Energenie EG-BA-AA4-01,5.0,NaN
3,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-aaa4-01/,Т-000088222,2024-11-07,В_наличии,Батарейка Energenie EG-BA-AAA4-01,5.0,NaN
4,https://hi-tech.md/televizory-i-elektronika/batareyki-i-zaryadnye-ustroystva/batareyka-energenie-eg-ba-cr2025-01/,Т-000088230,2024-11-07,В_наличии,Батарейка Energenie EG-BA-CR2025-01,5.0,NaN


****************************************************************************************************
таблица имеет 8630 сторок и 7 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                0
id                 0
date               0
availability       0
title              0
price              0
old_price       6665
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url              0.0
id               0.0
date             0.0
availability     0.0
title            0.0
price            0.0
old_price       77.2
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                     object
id                      object
date            datetime64[ns]
availability            object
title                   object
price                  float64
old_price              float64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8630 entries, 0 to 8629
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           8630 non-null   object        
 1   id            8630 non-null   object        
 2   date          8630 non-null   datetime64[ns]
 3   availability  8630 non-null   object        
 4   title         8630 non-null   object        
 5   price         8630 non-null   float64       
 6   old_price     1965 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 472.1+ KB


None

In [46]:
hai_group = hai_group.drop_duplicates(keep=False )

In [47]:
dfh = dfh.merge(hai_group, how='left', on='id')

обединение таблиц предварительного парсинга и таблицы групп товаров

In [48]:
h_df_new = dfh[(dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna())]

создание тыблицы с товарами, у которых нет данных о группах

In [49]:
dfh = dfh[~((dfh['_category'].isna()) | (dfh['_subcategory'].isna()) | (dfh['_group'].isna()) | (dfh['_subgroup'].isna()))]

отчистка таблицы парсинга от пропусков

In [50]:
if len(h_df_new) > 0:
    tqdm.pandas() 
    h_df_new[['_category','_subcategory','_group','_subgroup']] = h_df_new['url'].progress_apply(h_xxx)
else:
    pass

In [51]:
h_data = h_df_new[~((h_df_new['_category']=='err1') | (h_df_new['_category']=='err0'))].reset_index(drop=True)
h_data = h_data[['id', '_category','_subcategory','_group', '_subgroup']]
h_data = h_data.sort_values(['_category','_subcategory', '_group', '_subgroup'])

создание таблици с группами нового товара

In [52]:
hai_group = pd.concat([hai_group, h_data], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)
hai_group.to_csv('csv/ahay_group.csv', index=False)  

добавление в таблицу с группами товара новых данных

In [53]:
dfh = pd.concat([dfh, h_df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group', '_subgroup']).reset_index(drop=True)

объединение таблици со старыми и новыми товарами

In [54]:
dfh['error_group'] = 0
dfh.loc[dfh['_subgroup'].isna(), 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err0', 'error_group'] = 1
dfh.loc[dfh['_subgroup']=='err1', 'error_group'] = 1

Создание столбцов с флагом ошибки для товаров, для которых нет данных по остаткам хотябы в одном магазине

In [55]:
dfh.loc[dfh['error_group']==1 , ['_category',
       '_subcategory', '_group', '_subgroup']] = 'H_no_group'

удаление наличия товара в магазинах и наименовании группы если у товара есть ошибки в этих данны

In [56]:
dfh = dfh[['url', 'id', '_category', '_subcategory', '_group', '_subgroup', 'title',
               'date',   'price', 'old_price', 'availability', 'error_group', ]]

In [57]:
dfh['tir'] = 0

изменение порядка располажения столбцов

### Группировка товара

- разобраться с креслами в ногруп

In [58]:
dfh.loc[dfh['_category']=='H_товар_без_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_category']=='H_товар_по_акции', '_category'] = 'H_no_group'
dfh.loc[dfh['_subcategory']=='H_мультимедиа', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_акустические_системы', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_музыкальная_техника', '_group'] = 'H_акустика'
dfh.loc[dfh['_subcategory']=='H_умный_дом_с_яндекс','_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_яндекс_станции', '_group'] = 'H_акустика'
dfh.loc[dfh['_group']=='H_airpods', '_subcategory'] = 'H_аудиосистемы'
dfh.loc[dfh['_group']=='H_airpods', '_group'] = 'H_наушники'
dfh.loc[dfh['_group']=='H_apple_watch', '_group'] = 'H_часы_и_браслеты'
dfh.loc[dfh['_group']=='H_iphone', '_group'] = 'H_смартфоны'
dfh.loc[dfh['_group']=='H_ipad', '_group'] = 'H_планшеты'
dfh.loc[dfh['_group']=='H_macbook', '_group'] = 'H_ноутбуки'
dfh.loc[dfh['_group']=='H_imac', '_group'] = 'H_моноблоки'
dfh.loc[dfh['_group']=='H_часы_и_браслеты', '_subcategory'] = 'H_гаджеты'
dfh.loc[dfh['_group']=='H_смартфоны', '_subcategory'] = 'H_смартфоны_и_аксессуары'
dfh.loc[dfh['_group']=='H_планшеты', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_ноутбуки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_group']=='H_моноблоки', '_subcategory'] = 'H_готовые_решения'
dfh.loc[dfh['_subcategory']=='H_стиральные_и_сушильные_машины', '_group'] = 'H_стиральные_машины'
dfh.loc[dfh['_subgroup']=='H_сушильные_машины', '_group'] = 'H_сушильные_машины'
dfh.loc[dfh['_subgroup']=='H_наклейки', '_group'] = 'H_наклейки'
dfh.loc[dfh['_subgroup']=='H_подставки', '_group'] = 'H_подставки'
dfh.loc[dfh['_subgroup']=='H_замки', '_group'] = 'H_замки'
dfh.loc[dfh['_subgroup']=='H_usb_hub', '_group'] = 'H_usb_hub'
dfh.loc[dfh['_subgroup']=='H_сумки_и_чехлы', '_group'] = 'H_сумки_и_чехлы'
dfh.loc[dfh['_subgroup']=='H_защитные_стекла', '_group'] = 'H_защитные_стекла'
dfh.loc[dfh['_subcategory']=='H_мониторы','_group'] = 'H_мониторы'

In [59]:
for i in dfh['_category'].unique():
    t = dfh[dfh['_category']==i]
    print(i)
    print(t['_subcategory'].unique())
    print('-'*100)

H_бытовая_техника
['H_аксессуары_для_мелкой_бытовой_техники' 'H_климатическая_техника'
 'H_крупная_техника_для_кухни' 'H_мелкая_техника_для_кухни' 'H_минимойки'
 'H_приготовление_напитков' 'H_приготовление_пищи'
 'H_пылесосы_и_аксессуары' 'H_стиральные_и_сушильные_машины'
 'H_техника_karcher' 'H_техника_для_здоровья' 'H_техника_для_красоты'
 'H_товары_для_ухода_за_одеждой' 'H_установка_бытовой_техники']
----------------------------------------------------------------------------------------------------
H_компьютерная_техника
['H_аудиосистемы' 'H_гаджеты' 'H_apple' 'H_готовые_решения'
 'H_смартфоны_и_аксессуары' 'H_аксессуары' 'H_видеотехника'
 'H_игровые_приставки' 'H_комплектующие' 'H_крепления' 'H_мониторы'
 'H_накопители' 'H_оргтехника' 'H_периферийные_устройства'
 'H_проекторы_и_экраны' 'H_расходные_материалы' 'H_сетевое_оборудование'
 'H_электропитание']
----------------------------------------------------------------------------------------------------
H_мебель_текстиль
['H_мебел

In [60]:
for i in dfh['_subcategory'].unique():
    t = dfh[dfh['_subcategory']==i]
    print(i)
    print(t['_group'].unique())
    print('-'*100)

H_аксессуары_для_мелкой_бытовой_техники
['H_аксессуары_для_зубных_щеток' 'H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_климатическая_техника
['H_бойлеры' 'H_вентиляторы' 'H_комплектующие_для_климатической_техники'
 'H_кондиционеры' 'H_кондиционеры_кассетные' 'H_кондиционеры_мульти_сплит'
 'H_кондиционеры_промышленные' 'H_котлы_газовые' 'H_обогреватели'
 'H_отопительные_печи' 'H_увлажнители_комплексы']
----------------------------------------------------------------------------------------------------
H_крупная_техника_для_кухни
['H_варочные_поверхности' 'H_вытяжки' 'H_духовые_шкафы'
 'H_кронштейны_для_микроволновых_печей' 'H_микроволновые_печи'
 'H_морозильники_и_лари' 'H_плиты_газовые_и_электрические'
 'H_посудомоечные_машины' 'H_холодильники' 'H_электропечи_и_духовки']
----------------------------------------------------------------------------------------------------
H_мелкая_техника_для

просмотр групп товаров в подкатегориях

In [61]:
for i in dfh['_group'].unique():
    t = dfh[dfh['_group']==i]
    print(i)
    print(t['_subgroup'].unique())
    print('-'*100)

H_аксессуары_для_зубных_щеток
['H_аксессуары_для_зубных_щеток']
----------------------------------------------------------------------------------------------------
H_аксессуары_для_электрических_бритв
['H_аксессуары_для_электрических_бритв']
----------------------------------------------------------------------------------------------------
H_бойлеры
['H_бойлеры']
----------------------------------------------------------------------------------------------------
H_вентиляторы
['H_вентиляторы']
----------------------------------------------------------------------------------------------------
H_комплектующие_для_климатической_техники
['H_комплектующие_для_климатической_техники']
----------------------------------------------------------------------------------------------------
H_кондиционеры
['H_кондиционеры']
----------------------------------------------------------------------------------------------------
H_кондиционеры_кассетные
['H_кондиционеры_кассетные']
--------------------

## Объединение данных

In [62]:
df = pd.concat([dfh, dft], ignore_index=True)

In [63]:
df = df.drop_duplicates().reset_index(drop=True)

## Аналитика товаров

In [64]:
brend['names'] = brend['name'].apply(lower)
brend_up = brend.name.unique()
brend_low = brend.names.unique()

In [65]:
tqdm.pandas()
df['brend'] = df['title'].progress_apply(brends)

  0%|          | 0/20820 [00:00<?, ?it/s]

создаем столбец с брендами

In [66]:
df['action_sale'] = 0
df.loc[df['old_price'] > 0,'action_sale'] = 1

Создаем столбец с флагом акции

In [67]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [68]:
tqdm.pandas()
df['count_group'] = df['_group'].progress_apply(count_group)

  0%|          | 0/20820 [00:00<?, ?it/s]

создание столбца с количеством товара в группе

In [69]:
tqdm.pandas()
df['activ_count_group'] = df['_group'].progress_apply(count_group_presence)

  0%|          | 0/20820 [00:00<?, ?it/s]

создание столбца с количеством товара, который есть в наличии, в группе

In [70]:
tqdm.pandas()
df['dif_price'] = df['_group'].progress_apply(dif_price)

  0%|          | 0/20820 [00:00<?, ?it/s]

создам столбец с отношением самой дорогой цены в группе к самой дешевой.

In [71]:
df['group_prise'] = df.groupby('_group')['price'].transform(lambda x: pd.cut(x, bins = 4, labels=["бюджет", "стандарт", 'дорогой', "элитный"]))

создание столбца с ценовой категорией товара

In [72]:
df = df[['url', 'id', '_category', '_subcategory', '_group', 'brend', 
         'title', 'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise', 'dif_price',
         'availability', 'count_group', 'activ_count_group', 'error_group']]

In [73]:
df = df.drop_duplicates().reset_index(drop=True)

In [74]:
preprocessing(df)

таблица имеет следующий вид:


,url,id,_category,_subcategory,_group,brend,title,date,price,old_price,action_sale,sale,group_prise,dif_price,availability,count_group,activ_count_group,error_group
0,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,panasonic,насадка д/Зубная щетка Panasonic EW0940W830 2шт,2024-11-07,34.0,68.0,1,50.0,бюджет,NaN,В_наличии,7,0,0
1,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb10-2k-kids-frozen-1sht/,Т-000021198,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB10 Kids Frozen (1шт),2024-11-07,94.0,NaN,0,NaN,элитный,NaN,В_наличии,7,0,0
2,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb18-3d-white-1sht/,Т-000068433,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB18 3D White (1шт),2024-11-07,94.0,NaN,0,NaN,элитный,NaN,В_наличии,7,0,0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-braun-eb20rb-10-1sht/,Т-000068766,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB20 Precision Clean (1шт),2024-11-07,94.0,NaN,0,NaN,элитный,NaN,В_наличии,7,0,0
4,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zubnaya-schetka-braun-eb50-5/,Т-000070385,H_бытовая_техника,H_аксессуары_для_мелкой_бытовой_техники,H_аксессуары_для_зубных_щеток,braun,насадка д/Зубная щетка Braun EB50 Cross Action White (1шт),2024-11-07,94.0,NaN,0,NaN,элитный,NaN,В_наличии,7,0,0


****************************************************************************************************
таблица имеет 20820 сторок и 18 столбцов
****************************************************************************************************
в таблице обнаружено дублекатов:0
****************************************************************************************************
в таблице обнаружены пропуски в следующих столбцах:


url                      0
id                       0
_category                0
_subcategory             0
_group                   0
brend                    0
title                    0
date                     0
price                    5
old_price            16882
action_sale              0
sale                 16882
group_prise              5
dif_price             8632
availability             0
count_group              0
activ_count_group        0
error_group              0
dtype: int64

****************************************************************************************************
Эти пропуски составлябт следующее количество в процентах


url                   0.0
id                    0.0
_category             0.0
_subcategory          0.0
_group                0.0
brend                 0.0
title                 0.0
date                  0.0
price                 0.0
old_price            81.1
action_sale           0.0
sale                 81.1
group_prise           0.0
dif_price            41.5
availability          0.0
count_group           0.0
activ_count_group     0.0
error_group           0.0
dtype: float64

****************************************************************************************************
столбцы имеют следующие типы:


url                          object
id                           object
_category                    object
_subcategory                 object
_group                       object
brend                        object
title                        object
date                 datetime64[ns]
price                       float64
old_price                   float64
action_sale                   int64
sale                        float64
group_prise                category
dif_price                   float64
availability                 object
count_group                   int64
activ_count_group             int64
error_group                   int64
dtype: object

****************************************************************************************************
названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20820 entries, 0 to 20819
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   url                20820 non-null  object        
 1   id                 20820 non-null  object        
 2   _category          20820 non-null  object        
 3   _subcategory       20820 non-null  object        
 4   _group             20820 non-null  object        
 5   brend              20820 non-null  object        
 6   title              20820 non-null  object        
 7   date               20820 non-null  datetime64[ns]
 8   price              20815 non-null  float64   

None

## Выгрузка данных

In [75]:
df.to_csv('csv/hay_tir_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [77]:
df.columns

Index(['url', 'id', '_category', '_subcategory', '_group', 'brend', 'title',
       'date', 'price', 'old_price', 'action_sale', 'sale', 'group_prise',
       'dif_price', 'availability', 'count_group', 'activ_count_group',
       'error_group'],
      dtype='object')